# Clase 7: Minería de texto
En esta clase vamos a ver el tratamiento básico de texto, expresiones regulares, un modelo de tópicos __Latent Dirichlet Allocation (LDA)__.

Cubriremos los siguientes temas:
* Tratamiento de textos
* Preprocesamiento de textos
* WordClouds
* Document-Term Matrix
* LDA
* Interpretación de resultados
    * Palabras más importantes
    * Distribución de textos por tópicos
* Selección de modelo
* Visualización (LDAvis)

Para estudiar LDA, a parte del paper original, les recomendamos el siguiente video: https://www.youtube.com/watch?v=3mHy4OSyRf0&t=6s

La base de datos que trabajaremos será una muestra de noticias de [Reuters](https://www.reuters.com/) obtenida a través de webscrapping por Germán González.

1. ¿Qué significa que un problema sea de análisis no supervisado?
2. ¿Cómo evalúo uno de estos modelos?
3. ¿Que son datos no estructurados?
4. ¿Cómo entiende un computador los datos no estructurados?


In [ ]:
import pandas as pd 
import re # El paquete para tratar texto. Expresiones regulares
from sklearn.feature_extraction.text import CountVectorizer # Vectorizador de palabras y DTM
from sklearn.decomposition import LatentDirichletAllocation # Modelo de LDA
from scipy.sparse import csr_matrix # Para tratar Sparse Matrix
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data=pd.read_csv('reuters.csv') # CArgo los datos

In [ ]:
 # Explore el encabezado

In [ ]:
 # Exploro una noticia

## Preprocesamiento
* Tokenizar: Separar el texto en párrafos, frases, etc...
* Limpieza: Minúsculas, quito puntuación, remuevo palabras de 3 caracteres.
* Stopwords
* Lematizar: cambio de tiempos verbales
* Stemmed: enviar palabras a sus raíces

## Limpieza básica
Para ver más sobre expresiones regulares vea: https://www.w3schools.com/python/python_regex.asp

Puede practicar en: https://regex101.com/

In [ ]:
 # Busco las palabras que comienzan por f

### Ejercicios
Extraiga de una noticia cualquiera lo siguiente:
1. Los números. (piense en los decimales)
2. Los número que correspondan a porcentajes (vea la palabra percent).
4. Todas la palabras que comiencen por Mayúscula.
5. Todas la palabras que comiencen por Mayúscula pero el resto de caracteres estén en minúscula.
5. Extraiga todas las palabras que inicien por `r` (mayúscula o minúscula).

In [ ]:
# Esriba la solución aquí

In [ ]:
## Limpieza.
### Se usa los métodos de pandas que provienen de .str

# Envío a minúsculas
# Borro Puntuaciones
# Quito números
# Quito la palabra reuters
# Quito los dobles espacios
# Convierto minúsculas

In [ ]:
# Volvemos a ver la misma noticia

In [ ]:
# Ahora construiremos la matriz término-documento
 # máximo tamaño de vocabulario
# Al igual que un modelo, defino el objeto que construirá la matriz
# Aplico el objeto a un conjunto de textos
# Veo el vocabulario

In [ ]:
# Exploremos los stopwords

In [ ]:
 # Vuelvo de sparse a densa para explorarla
#Veo las primeras 5 filas
# Veo las dimensiones, a qué corresponden?

In [ ]:
# Exploramos la matriz término-documento

In [ ]:
 # Cuántas veces aparece year?

In [ ]:
# ¿Que tal si estudiamos las frecuencias de las palabras?


In [ ]:
# Grafique en barras las frecuencias

In [ ]:
!pip install wordcloud 

In [ ]:
from wordcloud import WordCloud #importo la función

In [ ]:
# Construyo el generador de la nube
 # Genero la nube
 # Despliego la imagen de la nube
# Para ver las gamas de colores vea: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
# Para ver más parámetros de la nube de palabras: https://amueller.github.io/word_cloud/auto_examples/index.html#example-gallery

## Modelo LDA

In [ ]:
 # Cuántos tópicos deseo
 # Construyo el objeto que es el modelo
 # Estimo el LDA

In [ ]:
# De que tma~no es el resultado?
 # Exploremos el resultado

In [ ]:
# Construyo la función que me ayuda a ver las palabras más importantes de cada tópico
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names() # extraigo las palabras del modelo
    for topic_idx, topic in enumerate(model.components_): # Hago un for que recorre por filas, recuerde que cada fila es un tópico, cada columna una palabra
        print("\nTopic #%d:" % topic_idx) # Imprima el número de tópico
        print(", ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) #Muestre las n palabras más importantes por orden 

In [ ]:
 # Veo las 15 palabras más importantes de cada tópico

In [ ]:
# Como se ven los documentos?
 # transformo la matrix de término-documento en tópico-documento
 # Qué indican las dimensiones?
 # Nombres de filas
 # Nombres de columnas


In [ ]:
# Exploremos la salida desde el punto de vista de documentos
 # Porque las filas suman 1?


In [ ]:
# Cómo se distribuye el documento promedio?


In [ ]:
# Creemos la pertenencia al tópicos


In [ ]:
# Construimos el histograma


## Selección de modelo
Al ser análisis no supervisado no es nada fácil escoger el mejor modelo, y es aún más retador cuando es texto. Tenemos una aproximación, la máxima verosimilitud

In [ ]:
%%time
# Juguemos con un hiper parámetro
likelihood=[]
values=[i for i in range(2,31,2)]
for i in values:
    modelo = LatentDirichletAllocation(n_components=i, max_iter=10,doc_topic_prior=0.1, topic_word_prior=0.1, n_jobs=-1,random_state=23) # Construyo el objeto que es el modelo
    modelo.fit(tf)
    likelihood.append(modelo.score(tf))
    print(i)

In [ ]:
# Visualizamos
plt.figure(figsize=(6,6))
plt.plot(values, likelihood)
plt.xlabel('Número de tópicos')
plt.ylabel('log-likelihood')

## Visualización del LDA
LDAvis es un paquete para la visualización para interpretar más fácilmente el LDA. Vea el paper [aquí](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf), en este agregan un nuevo parámetro para la interpretació, $\lambda \in [0,1]$, el cual pondera la importancia de una palabra dentro del tópico por la unicidad de la palabra a lo largo de los tópicos. 
* $\lambda\rightarrow1$: este es el caso original, permite que la palabra sea repetida a lo largo de los tópicos.
* $\lambda\rightarrow0$: este caso cambia el rqanking de importancia, dándole mayor importancia a las palabras que sean únicas del tópico, es decir que no aparezcan casi en los demás.


In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis # Paquete que crea la visualización
from pyLDAvis import sklearn as sklearnlda

In [ ]:
 # Preparo el modelo y sus resultados para la visualización
# Guardo la visualización como html

In [ ]:
 # Lo visualizo dentro del notebook